In [1]:
from endless_line.data_utils.dataloader import DataLoader

In [10]:
data = DataLoader(load_all_files=True)

In [11]:
data.load_file("attendance.csv")



,USAGE_DATE,FACILITY_NAME,attendance
0,2018-06-01,PortAventura World,46804
1,2018-06-01,Tivoli Gardens,20420
2,2018-06-02,PortAventura World,57940
3,2018-06-02,Tivoli Gardens,29110
4,2018-06-03,PortAventura World,44365
...,...,...,...
2362,2022-07-24,Tivoli Gardens,26798
2363,2022-07-25,PortAventura World,53764
2364,2022-07-25,Tivoli Gardens,27061
2365,2022-07-26,PortAventura World,26391
